# 자동화된 Machine Learning 설명 검토

기계 학습이 사용되는 분야가 점점 늘어남에 따라 모델에서 생성하는 예측이 사회의 여러 분야에 미치는 영향력도 더욱 커지고 있습니다. 예를 들어 은행에서 대출을 결정하거나 의사가 치료 우선 순위를 결정하는 과정에서 기계 학습 모델의 중요도가 갈수록 높아지고 있습니다. 이에 따라 모델을 해석하고 설명하는 기능도 더욱 중요해졌습니다. 이러한 기능을 활용하면 기계 학습 모델이 생성하는 예측의 근거를 설명하고 타당성을 증명할 수 있으며, 모델의 부적절한 바이어스를 파악할 수 있습니다.

자동화된 기계 학습을 사용하여 모델을 학습시킬 때는 기능 중요도 설명을 생성할 수 있습니다. 이 설명에는 각 기능이 레이블 예측에 영향을 주는 범위가 수치로 표시됩니다. 이 랩에서는 자동화된 Machine Learning 실험에서 생성된 설명을 살펴봅니다.

## 작업 영역에 연결

가장 먼저 해야 하는 작업은 Azure ML SDK를 사용하여 작업 영역에 연결하는 것입니다.

> **참고**: 이전 연습을 완료한 후 Azure 구독으로 인증된 세션이 만료된 경우 다시 인증하라는 메시지가 표시됩니다.

In [ ]:
import azureml.core
from azureml.core import Workspace

# 저장된 구성 파일에서 작업 영역 로드
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 자동화된 Machine Learning 실험 실행

이 랩에서는 시간을 절약하기 위해 3회만 반복되는 자동화된 기계 학습 실험을 실행하겠습니다.

**model_explainability** 구성 옵션은 **True**로 설정됩니다.

In [ ]:
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Dataset

cluster_name = "gmalc-aml-clust" # Change to your compute cluster name

# 학습용 데이터 준비
default_ds = ws.get_default_datastore()
if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # /data에서 당뇨병 CSV 파일 업로드
                        target_path='diabetes-data/', # 데이터 저장소의 폴더 경로에 해당 파일 저장
                        overwrite=True, # 이름이 같은 기존 파일 바꾸기
                        show_progress=True)

    #데이터 저장소의 경로에서 테이블 형식 데이터 세트 만들기(시간이 다소 걸릴 수 있음)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # 테이블 형식 데이터 세트 등록
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')
train_data = ws.datasets.get("diabetes dataset")

# 자동 ML 구성
automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target='local',
                             enable_local_managed=True,
                             training_data = train_data,
                             n_cross_validations = 2,
                             label_column_name='Diabetic',
                             iterations=3,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=3,
                             featurization='off',
                             model_explainability=True # 기능 중요도 생성
                             )

# 자동 ML 실험 실행
print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'diabetes_automl')
automl_run = automl_experiment.submit(automl_config)
automl_run.wait_for_completion(show_output=True)
RunDetails(automl_run).show()

## 기능 중요도 확인

위의 위젯에서 실험이 완료되면 최적 결과를 생성한 실험을 클릭하여 해당 세부 정보를 살펴봅니다. 그런 다음 시각화의 맨 아래로 스크롤하여 상대적 기능 중요도를 확인합니다.

**ExplanationClient** 클래스를 사용하면 실험에서 생성된 최적 모델의 기능 중요도를 확인할 수도 있습니다.

In [ ]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient
from azureml.core.run import Run

# 최고의 모델 설명 실행이 완료될 때까지 기다립니다.
model_explainability_run_id = automl_run.get_properties().get('ModelExplainRunId')
print(model_explainability_run_id)
if model_explainability_run_id is not None:
    model_explainability_run = Run(experiment=automl_experiment, run_id=model_explainability_run_id)
    model_explainability_run.wait_for_completion(show_output=True)

# 최적 모델(출력의 두 번째 항목) 가져오기
best_run, fitted_model = automl_run.get_output()

# 기능 설명 가져오기
client = ExplanationClient.from_run(best_run)
engineered_explanations = client.download_model_explanation()
feature_importances = engineered_explanations.get_feature_importance_dict()

# 전체 기능 중요도
print('Feature\tImportance')
for key, value in feature_importances.items():
    print(key, '\t', value)

## Azure Machine Learning Studio에서 모델 설명 확인

실험 실행이 완료되면 위젯의 링크를 클릭하여 Azure Machine Learning Studio에서 실행을 확인하고 **설명** 탭을 표시합니다. 그런 후에 다음 단계를 수행합니다

1. 자동화된 Machine Learning 실행에서 생성된 설명자를 선택합니다.
2. 전반적인 글로벌 기능 중요도가 표시되는 **글로벌 중요도** 차트를 확인합니다.
3. *모집단*, *바이올린* 또는 *상자* 그림에 테스트 데이터의 각 데이터 요소가 표시되는 **요약 중요도** 차트를 확인합니다.
4. 개별 요소를 선택하여 선택한 데이터 요소의 개별 예측 **로컬 기능 중요도**를 확인합니다.

> **추가 정보**: 자동화된 Machine Learning에 대한 자세한 내용은 [Azure ML 설명서](https://docs.microsoft.com/azure/machine-learning/how-to-machine-learning-interpretability-automl)를 참조하세요.